In [4]:
# -----------------------------
# Real-Time Face Mask Detection (Professional Version)
# -----------------------------

import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import time

# -----------------------------
# 1. Load Pretrained Model
# -----------------------------
MODEL_PATH = r"C:\Users\PMYLS\Downloads\mask_detector.model"
mask_model = load_model(MODEL_PATH)

# -----------------------------
# 2. Load Haar Cascade for face detection
# -----------------------------
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# -----------------------------
# 3. Preprocessing function
# -----------------------------
def preprocess_face(face_img):
    face = cv2.resize(face_img, (224, 224))
    face = img_to_array(face)
    face = face / 255.0
    face = np.expand_dims(face, axis=0)
    return face

# -----------------------------
# 4. Draw confidence bar
# -----------------------------
def draw_confidence_bar(frame, x, y, w, h, mask_prob, no_mask_prob):
    bar_x = x
    bar_y = y + h + 10
    bar_width = w
    bar_height = 10

    # Mask probability bar
    mask_width = int(bar_width * mask_prob)
    cv2.rectangle(frame, (bar_x, bar_y), (bar_x + mask_width, bar_y + bar_height), (0, 255, 0), -1)

    # No-mask probability bar
    no_mask_width = int(bar_width * no_mask_prob)
    cv2.rectangle(frame, (bar_x, bar_y + bar_height), (bar_x + no_mask_width, bar_y + 2*bar_height), (0, 0, 255), -1)

# -----------------------------
# 5. Start Webcam
# -----------------------------
cap = cv2.VideoCapture(0)
fps_start_time = time.time()
frame_count = 0

# For simple face tracking
prev_faces = []

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60))

    faces_input = []
    coords = []

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        face_input = preprocess_face(face_img)
        faces_input.append(face_input)
        coords.append((x, y, w, h))

    if len(faces_input) > 0:
        faces_batch = np.vstack(faces_input)
        preds = mask_model.predict(faces_batch, batch_size=32)

        for pred, (x, y, w, h) in zip(preds, coords):
            mask_prob, no_mask_prob = pred
            confidence = max(mask_prob, no_mask_prob)

            # Determine label with low-confidence check
            if confidence < 0.6:
                label = "Uncertain"
                color = (0, 255, 255)  # yellow for uncertainty
            else:
                label = "Mask" if mask_prob > no_mask_prob else "No Mask"
                color = (0, 255, 0) if mask_prob > no_mask_prob else (0, 0, 255)

            # Draw bounding box
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

            # Draw label with background
            cv2.rectangle(frame, (x, y-25), (x+w, y), color, cv2.FILLED)
            text = f"{label}: {confidence*100:.1f}%"
            cv2.putText(frame, text, (x+5, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)

            # Draw confidence bars
            draw_confidence_bar(frame, x, y, w, h, mask_prob, no_mask_prob)

    # -----------------------------
    # Display FPS
    # -----------------------------
    frame_count += 1
    fps_end_time = time.time()
    fps = frame_count / (fps_end_time - fps_start_time)
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    cv2.imshow("Professional Face Mask Detector", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 69ms/step
